In [3]:
import configparser
import os

config = configparser.RawConfigParser()
config.read('keys.config')
os.environ["ACTIVELOOP_TOKEN"] = config.get('keys', 'active_loop_key')
os.environ["OPENAI_API_KEY"] = config.get('keys', 'open_ai_key')


In [1]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate

# Load Sample
f = open('sample_espresso.txt', 'r')
rows = f.readlines()

sample = []
for r  in rows:
    if "Category:" in r: # New Category
        sample.append({"query": r})
    else:
        if sample[-1].get("answer", None) is None:
            sample[-1]["answer"] = ""
        
        sample[-1]["answer"] += r

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following is a template for Energy-related News, grouped by category.\
    Use this template as a sample to generate new Energy-related News by category.
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=sample,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

/opt/homebrew/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.11) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [6]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

# load the model
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1.0)

chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)
chain.run("Create 3 new headlines for Category Geopolitics?")

"1. OPEC members convene for critical meeting to discuss global energy policies and market stability.\n2. Iran's enriched uranium levels rise as international inspectors report reinstallation of monitoring equipment.\n3. Geopolitical tensions rise as OPEC meeting coincides with Iran's nuclear developments."